In [1]:
import numpy as np
import csv
#import pandas as pd
import sys
import time
import math


with open( "Book1.csv") as csvfile:
    csvreader = csv.reader(csvfile)
    #next(csvreader, None)  # if 1st row is list then active
    dataset = list(csvreader)
no_of_user  =len(dataset)
no_of_item = len(dataset[0])

dataset_matrix = np.zeros((no_of_user,no_of_item))
for i in range (no_of_user):
    for j in range(no_of_item):
        dataset_matrix[i][j] = dataset[i][j]

Actual_matrix = np.zeros((no_of_user,no_of_item))
predected_matrix = np.zeros((no_of_user,no_of_item))
rating_matrix = np.zeros((no_of_user,no_of_item))
for i in range (no_of_user):
    for j in range(no_of_item):
        Actual_matrix[i][j] = dataset[i][j]
        
#print(dataset,no_of_user,no_of_item)
 


for user_index in range(no_of_user):
    for item_index in range (no_of_item):
            dataset_matrix[user_index][item_index] = 0
   # print("Number of User",no_of_user,'\n',"NUmber of Item",no_of_item)
   # print("Original dataset_matrix or User Matrix",'\n',dataset_matrix)
          
        
        # --------------------USER PROFILE--------------------------
            #print(dataset_matrix)
            user_profile = np.zeros(no_of_user)

            for i in range(no_of_user):
                count = 0
                sums = 0
                for j in range (no_of_item):
                    sums = sums + int(dataset_matrix[i][j])
                    if(int(dataset_matrix[i][j]) != 0):
                        count = count+1
                if(count!=0):
                    user_profile[i] = sums/count 
                print("User profile for",i,'th user',user_profile[i])



             # Item Profile ,  Averge rating of each item by different user  
            dataset_matrix_transpose = np.zeros((no_of_item,no_of_user))
            dataset_matrix_transpose = dataset_matrix.T
           # print(" Item Matrix",'\n',dataset_matrix_transpose)

            item_profile = np.zeros(no_of_user)

            for i in range(no_of_item):

                count = 0
                sums = 0

                for j in range (no_of_user):
                    sums = sums + int(dataset_matrix_transpose[i][j])
                    if(int(dataset_matrix_transpose[i][j]) != 0):
                        count = count+1
                if(count!=0):
                    item_profile[i] = sums/count
                print("Item profile for",i,'th Item',item_profile[i])



            # User Tendency
            
            user_tendency = np.zeros(no_of_user)
            for i in range(no_of_user):
                count = 0
                sums = 0
                for j in range (no_of_item):
                    if(int(dataset_matrix[i][j]) != 0):
                        sums = sums + int(dataset_matrix[i][j]) - item_profile[i]
                        count = count+1
                if(count!=0):
                    user_tendency[i] = sums/count
                print("User tendency for",i,'th user',user_tendency[i])
            

            # Item Tendency
            
            item_tendency = np.zeros(no_of_item)
           # print("Datset item matrix",dataset_matrix_transpose)
            for i in range(no_of_item):
                count = 0
                sums = 0
                for j in range (no_of_user):
                    if(int(dataset_matrix_transpose[i][j]) != 0):
                       # print(dataset_matrix_transpose[i][j])
                        sums = sums + int(dataset_matrix_transpose[i][j]) - round(user_profile[i],3)
                        count = count+1
                        sums = round(sums,3)
                        #print("sum",sums)
                #print(count,item_profile[i])
                if(count!=0):
                    item_tendency[i] = sums/count
               # item_tendency[i] = round(item_tendency[i],3)
                print("item tendency for",i,'th user',round(item_tendency[i],3))


            # Case 1  User Profile and Item Profile is Positive
            beta =0.5
            
            if(user_tendency[user_index] >= 0 and item_tendency[item_index] >= 0):
                # ---------------------FIND MAXIMUM ------------------------
                
                if(item_tendency[item_index] + user_profile[user_index] > user_tendency[user_index] + item_profile[item_index] ):
                    rating = item_tendency[item_index] + user_profile[user_index]
                    #print("rating of user",rating)
                else:
                    rating = user_tendency[user_index] + item_profile[item_index]
                    #print("rating of user",rating)
            
            # Case 2 User Profile and Item Progile is Negative    
            
            elif(user_tendency[user_index] < 0 and item_tendency[item_index] < 0):
                 # ---------------------FIND MINIMUM ------------------------
                if(item_tendency[item_index] + user_profile[user_index] > user_tendency[user_index] + item_profile[item_index] ):
                    rating = user_tendency[user_index] + item_profile[item_index] 
                    #print("rating of user",rating)
                    #predected_matrix[user_index][item_index] = rating
                else:
                    rating = item_tendency[item_index] + user_profile[user_index]
                    #print("rating of user",rating)
                    #predected_matrix[user_index][item_index] = rating
            
            # Case 3 User Profile is Positive and Item Progile is Negative and Item mean is leass than User Mean       
            elif(user_tendency[user_index] >= 0 and item_tendency[item_index] <= 0 and item_profile[item_index]< user_profile[user_index] ):
                rating = min(max(item_profile[item_index],(user_tendency[user_index]+item_profile[item_index])*beta + (item_tendency[item_index] + user_profile[user_index])*(1-beta)),user_profile[user_index])
                #print("rating of user",rating)
                #predected_matrix[user_index][item_index] = rating
            
            # Case 4 User Profile is Negative and Item Progile is Positive and Item Mean is greater than User mean
            elif(user_tendency[user_index] <= 0 and item_tendency[item_index] >= 0 and item_profile[item_index]> user_profile[user_index] ):
                rating = min(max(user_profile[item_index],(user_tendency[user_index]+item_profile[item_index])*beta + (item_tendency[item_index] + item_profile[user_index])*(1-beta)),user_profile[user_index])
               # print("rating of user",rating)
                #predected_matrix[user_index][item_index] = rating
             
            # Case 5 Otherwise
            else:
                rating = item_profile[item_index]*beta + user_profile[user_index]
                #print("rating of user",rating)
            rating_matrix[user_index][item_index] = rating
            if(rating<5):
                predected_matrix[user_index][item_index] = rating
            else:
                 predected_matrix[user_index][item_index] = 5
            dataset_matrix[user_index][item_index]  = Actual_matrix[user_index][item_index]
           


            
print("rating_matrix",'\n',rating_matrix)

print('\n\n')            
print("predected_matrix",'\n',predected_matrix)
print('\n\n')

# ------------------- FIND RELEVANT ITEM FOR EACH USER---------------

Relevant_item =np.zeros((no_of_user))
for user_index in range(no_of_user):
    count =0
    for item_index in range(no_of_item):
        if(Actual_matrix[user_index][item_index] > 3.5):
            count = count +1
           
    Relevant_item[user_index] = count
    
#print(Relevant_item)

Rec_Rel =np.zeros((no_of_user))
for user_index in range(no_of_user):
    count =0
    for item_index in range(no_of_item):
        if(Actual_matrix[user_index][item_index] > 3.5 and predected_matrix[user_index][item_index] > 3.5):
            count = count +1
           
    Rec_Rel[user_index] = count
    print("Recommended and Relevant",'\n',Rec_Rel)
Rec =np.zeros((no_of_user))
for user_index in range(no_of_user):
    count =0
    for item_index in range(no_of_item):
        if(predected_matrix[user_index][item_index] > 3.5):
            count = count +1
           
    Rec[user_index] = count 
    print("Recommended",'\n',Rec)

Precision = sum(Rec_Rel)/sum(Rec)
print("Over all Precision: ",Precision)

count =0
add =0
for user_index in range(no_of_user):
   
    for item_index in range(no_of_item):
        if(Actual_matrix[user_index][item_index]!=0):
            add = add + abs(predected_matrix[user_index][item_index] - Actual_matrix[user_index][item_index])
            count = count +1
            
Mae = add/count
print('\n\n')
print("Mean Absolute Error:" ,Mae)

for user_index in range(no_of_user):
   
    for item_index in range(no_of_item):
        if(Actual_matrix[user_index][item_index]!=0):
            add = add + abs(predected_matrix[user_index][item_index] - Actual_matrix[user_index][item_index]) **2
            count = count +1
add = add/count           
Rmse = np.sqrt(add)

print("Root mean squared Error:" ,Rmse)

Recall = sum(Rec_Rel)/sum(Relevant_item)
print("Recall:" ,Recall)

F1measure  = (2 * Precision * Recall)/(Precision +Recall)
print(" F1measurealue :",F1measure)

User profile for 0 th user 4.0
User profile for 1 th user 4.0
User profile for 2 th user 3.5
User profile for 3 th user 1.5
User profile for 4 th user 4.0
Item profile for 0 th Item 3.75
Item profile for 1 th Item 2.8
Item profile for 2 th Item 4.0
Item profile for 3 th Item 4.0
User tendency for 0 th user 0.25
User tendency for 1 th user 1.2000000000000002
User tendency for 2 th user -0.5
User tendency for 3 th user -2.5
User tendency for 4 th user 4.0
item tendency for 0 th user -0.25
item tendency for 1 th user -1.2
item tendency for 2 th user 0.5
item tendency for 3 th user 2.5
User profile for 0 th user 4.333333333333333
User profile for 1 th user 4.0
User profile for 2 th user 3.5
User profile for 3 th user 1.5
User profile for 4 th user 4.0
Item profile for 0 th Item 3.8
Item profile for 1 th Item 2.75
Item profile for 2 th Item 4.0
Item profile for 3 th Item 4.0
User tendency for 0 th user 0.5333333333333335
User tendency for 1 th user 1.25
User tendency for 2 th user -0.5
User